# 시총 top10 회사 뽑기

In [ ]:
import yfinance as yf
import pandas as pd

# S&P 500 Ticker 리스트
sp500_tickers = ["AAPL", "MSFT", "AMZN", "GOOGL", "META", "NVDA", "BRK.B", "TSLA", "JNJ", "JPM", "V", "XOM"]

# 날짜 범위 설정
start_date = "2016-12-01"
end_date = "2022-03-31"

# 결과 저장
market_cap_data = []

for ticker in sp500_tickers:
    print(f"Processing {ticker}...")
    
    # Yahoo Finance에서 데이터 가져오기
    stock_data = yf.download(ticker, start=start_date, end=end_date, interval="1mo")
    
    # 월말 종가와 Outstanding Shares 추정 (Yahoo Finance API에서 outstandingShares 가져오기)
    ticker_info = yf.Ticker(ticker).info
    stock_data["Market_Cap"] = stock_data["Close"] * ticker_info.get("sharesOutstanding")
    
    # 필요한 컬럼만 저장
    stock_data = stock_data[["Market_Cap"]]
    stock_data["Ticker"] = ticker
    market_cap_data.append(stock_data)

# 모든 데이터를 하나로 합치기
market_cap_df = pd.concat(market_cap_data)

# 월별 Top 10 Market Cap 추출
top10_market_cap = (
    market_cap_df.reset_index()
    .groupby("Date")
    .apply(lambda x: x.nlargest(10, "Market_Cap"))
    .reset_index(drop=True)
)

# 결과 저장 및 출력
top10_market_cap.to_csv("top10_sp500_market_cap.csv", index=False)
print(top10_market_cap)


In [11]:
! pip install yfinance

     ---------------------------------------- 0.0/948.2 kB ? eta -:--:--
     -------------------------------------- 948.2/948.2 kB 5.5 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for peewee: filename=peewee-3.17.8-py3-none-any.whl size=139012 sha256=5b772fee2e5134ca0f5d5be24d2b7382923bcd947a9161382fc06e24803ca4be
  Stored in directory: c:\users\dabin\appdata\local\pip\cache\wheels\8f\65\34\456800445efeafb05164fe95285c70e81ba1d96bae30f43917
Successfully built peewee


In [ ]:
import yfinance as yf
import pandas as pd

# S&P 500 전체 티커 가져오기
sp500_tickers = pd.read_html("https://en.wikipedia.org/wiki/List_of_S%26P_500_companies")[0]["Symbol"].tolist()

# 날짜 범위 설정
start_date = "2017-01-01"
end_date = "2022-04-01"

# 결과 저장
market_cap_data = []

# 모든 티커에 대해 데이터 수집
for ticker in sp500_tickers:
    try:
        print(f"Processing {ticker}...")
        
        # Yahoo Finance에서 데이터 가져오기
        stock_data = yf.download(ticker, start=start_date, end=end_date, interval="1mo")
        
        # 종가(Close)와 시가총액 계산
        ticker_info = yf.Ticker(ticker).info
        shares_outstanding = ticker_info.get("sharesOutstanding")  # 발행 주식 수
        
        # 발행 주식 수가 없으면 다음 티커로 넘어감
        if shares_outstanding is None:
            print(f"Skipping {ticker} (missing sharesOutstanding)")
            continue
        
        stock_data["Market_Cap"] = stock_data["Close"] * shares_outstanding
        
        # 필요한 컬럼 추가 및 저장
        stock_data = stock_data[["Market_Cap"]]
        stock_data["Ticker"] = ticker
        market_cap_data.append(stock_data)
    
    except Exception as e:
        print(f"Error processing {ticker}: {e}")

# 모든 데이터를 하나로 합치기
market_cap_df = pd.concat(market_cap_data)

# 월별 Top 10 Market Cap 추출
top10_market_cap = (
    market_cap_df.reset_index()
    .groupby("Date")
    .apply(lambda x: x.nlargest(10, "Market_Cap"))
    .reset_index(drop=True)
)

# 결과 저장 및 출력
top10_market_cap.to_csv("top10_sp500_dynamic_market_cap.csv", index=False)
print(top10_market_cap)


In [2]:
import pandas as pd
marketcap = pd.read_csv('C:/Users/DaBin/Desktop/대학원/2024-2/통계계산특론/PROJECT/top10_sp500_dynamic_market_cap.csv')
marketcap['Ticker']

0       NaN
1      MSFT
2      AAPL
3      AMZN
4       XOM
       ... 
626    GOOG
627    NVDA
628    META
629     UNH
630     JNJ
Name: Ticker, Length: 631, dtype: object

In [3]:
marketcap['Ticker'].nunique()

17

In [5]:
marketcap['Ticker'].unique()[1:]

array(['MSFT', 'AAPL', 'AMZN', 'XOM', 'META', 'JNJ', 'GOOGL', 'JPM', 'T',
       'GOOG', 'WMT', 'PG', 'BA', 'V', 'UNH', 'TSLA', 'NVDA'],
      dtype=object)

In [38]:
from IPython.display import display

# 데이터프레임 출력
display(pd.DataFrame(grouped))

,year,month,Ticker
0,2017.0,1.0,"[MSFT, AAPL, AMZN, XOM, META, JNJ, GOOGL, JPM,..."
1,2017.0,2.0,"[AAPL, MSFT, AMZN, XOM, META, JNJ, JPM, GOOGL,..."
2,2017.0,3.0,"[AAPL, MSFT, AMZN, XOM, META, JNJ, GOOGL, JPM,..."
3,2017.0,4.0,"[AAPL, MSFT, AMZN, XOM, META, JNJ, GOOGL, GOOG..."
4,2017.0,5.0,"[AAPL, AMZN, MSFT, XOM, META, JNJ, GOOGL, GOOG..."
...,...,...,...
58,2021.0,11.0,"[AAPL, MSFT, AMZN, TSLA, GOOGL, NVDA, GOOG, ME..."
59,2021.0,12.0,"[AAPL, MSFT, AMZN, TSLA, GOOGL, GOOG, META, NV..."
60,2022.0,1.0,"[AAPL, MSFT, AMZN, TSLA, GOOGL, GOOG, META, NV..."
61,2022.0,2.0,"[AAPL, MSFT, AMZN, TSLA, GOOGL, GOOG, NVDA, ME..."


In [35]:
marketcap['month'] = pd.to_datetime(marketcap['Date']).dt.month
marketcap['year'] = pd.to_datetime(marketcap['Date']).dt.year

marketcap = pd.DataFrame(marketcap)

# 그룹화 및 고유 값 추출
grouped = marketcap.groupby(['year', 'month'])['Ticker'].unique().reset_index()

# 결과 출력

grouped

,year,month,Ticker
0,2017.0,1.0,"[MSFT, AAPL, AMZN, XOM, META, JNJ, GOOGL, JPM,..."
1,2017.0,2.0,"[AAPL, MSFT, AMZN, XOM, META, JNJ, JPM, GOOGL,..."
2,2017.0,3.0,"[AAPL, MSFT, AMZN, XOM, META, JNJ, GOOGL, JPM,..."
3,2017.0,4.0,"[AAPL, MSFT, AMZN, XOM, META, JNJ, GOOGL, GOOG..."
4,2017.0,5.0,"[AAPL, AMZN, MSFT, XOM, META, JNJ, GOOGL, GOOG..."
...,...,...,...
58,2021.0,11.0,"[AAPL, MSFT, AMZN, TSLA, GOOGL, NVDA, GOOG, ME..."
59,2021.0,12.0,"[AAPL, MSFT, AMZN, TSLA, GOOGL, GOOG, META, NV..."
60,2022.0,1.0,"[AAPL, MSFT, AMZN, TSLA, GOOGL, GOOG, META, NV..."
61,2022.0,2.0,"[AAPL, MSFT, AMZN, TSLA, GOOGL, GOOG, NVDA, ME..."


In [ ]:
import pandas as pd

data = pd.read_csv('C:/Users/DaBin/Desktop/대학원/2024-2/통계계산특론/PROJECT/data/최종데이터/top10_market_cap.csv') #top10의 market cap
firm = pd.read_csv("C:/Users/oi236/Downloads/scaled_firm_new (2).csv")

In [72]:
# 2017년부터 2023년까지의 데이터만 필터링
filtered_firm_data_2017_2023 = firm[
    (firm['year'] >= 2017) & (firm['year'] <= 2023)
]

# 결과 확인
filtered_firm_data_2017_2023=filtered_firm_data_2017_2023[['ticker','month','year','ret']]

In [67]:
filtered_firm_data_2017_2023

,ticker,month,year,ret
239,AAPL,1.0,2017.0,0.047746
240,AAPL,2.0,2017.0,0.133581
241,AAPL,3.0,2017.0,0.048690
242,AAPL,4.0,2017.0,-0.000070
243,AAPL,5.0,2017.0,0.067804
...,...,...,...,...
105487,BX,5.0,2021.0,0.047237
105488,BX,6.0,2021.0,0.048236
105489,BX,7.0,2021.0,0.193844
105490,BX,8.0,2021.0,0.090744


In [70]:
filtered_firm_data=filtered_firm_data_2017_2023

In [78]:
import pandas as pd
import numpy as np

# 초기 투자 금액
initial_investment = 10000

# 투자 시뮬레이션 데이터프레임 생성
investment_results = []

# 2017년 1월부터 2022년 2월까지 월별로 루프
start_year = 2017
start_month = 1
end_year = 2022
end_month = 2

# 투자 금액 초기화
current_investment = initial_investment

# 월별 반복 루프
for year in range(start_year, end_year + 1):
    for month in range(1, 13):
        # 종료 조건
        if year == end_year and month > end_month:
            break
        
        # data에서 해당 연도와 월의 상위 티커 필터링
        filtered_data = data[(data['year'] == year) & (data['month'] == month)]

        # 해당 월에 투자할 티커 목록 가져오기
        tickers = filtered_data['Ticker'].tolist()
        

        # filtered_firm_data에서 해당 연도와 월에 대한 수익률 데이터 필터링
        returns_data = filtered_firm_data[
            (filtered_firm_data['year'] == year) &
            (filtered_firm_data['month'] == month) &
            (filtered_firm_data['ticker'].isin(tickers))
        ]

        # 만약 해당 월에 투자할 티커 데이터가 없으면 넘어감
        if returns_data.empty:
            investment_results.append({
                'year': year,
                'month': month,
                'investment': current_investment,
                'return_rate': None,
                'updated_investment': current_investment
            })
            continue

        # 투자 금액을 티커 개수만큼 균등 분배
        num_tickers = len(returns_data)
        investment_per_ticker = current_investment / num_tickers

        # 각 티커의 수익률 적용 후 총 투자 금액 계산
        returns_data['investment_value'] = investment_per_ticker * (1 + returns_data['ret'])
        updated_investment = returns_data['investment_value'].sum()

        # 수익률 계산
        return_rate = (updated_investment - current_investment) / current_investment * 100

        # 결과 저장
        investment_results.append({
            'year': year,
            'month': month,
            'investment': current_investment,
            'return_rate': return_rate,
            'updated_investment': updated_investment
        })

        # 현재 투자 금액 업데이트
        current_investment = updated_investment

# 결과를 데이터프레임으로 변환
investment_results_df = pd.DataFrame(investment_results)

# 결과 출력
investment_results_df




C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_32420\2042068210.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  returns_data['investment_value'] = investment_per_ticker * (1 + returns_data['ret'])
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_32420\2042068210.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  returns_data['investment_value'] = investment_per_ticker * (1 + returns_data['ret'])
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_32420\2042068210.py:56: SettingWit

,year,month,investment,return_rate,updated_investment
0,2017,1,10000.000000,2.100172,10210.017249
1,2017,2,10210.017249,3.583504,10575.893655
2,2017,3,10575.893655,2.601595,10851.035603
3,2017,4,10851.035603,2.760260,11150.552424
4,2017,5,11150.552424,7.314374,11966.145553
...,...,...,...,...,...
57,2021,10,40365.589019,11.693560,45085.763242
58,2021,11,45085.763242,3.310046,46578.122680
59,2021,12,46578.122680,0.976469,47032.943470
60,2022,1,47032.943470,-5.992797,44214.354719


# Model Based invesement

In [3]:
import pandas as pd
data = pd.read_csv('C:/Users/DaBin/Desktop/대학원/2024-2/통계계산특론/PROJECT/result/S_PCA58F_PCA50E_pred.csv')
filtered_firm_data = pd.read_csv('C:/Users/DaBin/Desktop/대학원/2024-2/통계계산특론/PROJECT/data/firm_new.csv')


In [6]:
# 월별 top10 선택
top10_monthly = (
    data.groupby(['year', 'month'])  # year와 month를 기준으로 그룹화
    .apply(lambda x: x.nlargest(10, 'ret'))  # 'ret' 기준으로 상위 10개 선택
    .reset_index(drop=True)  # 그룹화 결과의 인덱스를 초기화
)

# 결과 출력
top10_monthly

C:\Users\DaBin\AppData\Local\Temp\ipykernel_15972\2381266169.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.nlargest(10, 'ret'))  # 'ret' 기준으로 상위 10개 선택


,ret,year,month,ticker
0,0.101890,2017.0,1.0,WTW
1,0.073279,2017.0,1.0,EQIX
2,0.063190,2017.0,1.0,ULTA
3,0.056740,2017.0,1.0,ROP
4,0.055307,2017.0,1.0,SBAC
...,...,...,...,...
625,0.065048,2022.0,3.0,FTNT
626,0.059239,2022.0,3.0,ENPH
627,0.057910,2022.0,3.0,STLD
628,0.056977,2022.0,3.0,CNP


## 1. COR DATA

### 1.1 예측 수익률 데이터 + 실제 회사의 수익률 데이터 가져오기

In [6]:
import pandas as pd

data = pd.read_csv('C:/Users/DaBin/Desktop/대학원/2024-2/통계계산특론/PROJECT/result/COR30F_PCA50E_pred.csv')
filtered_firm_data = pd.read_csv('C:/Users/DaBin/Desktop/대학원/2024-2/통계계산특론/PROJECT/data/firm_new.csv')

### 1.2 월별 예측 수익률 top10 선택

In [8]:
# 월별 수익률 top10 선택
COR_top10_monthly = (
    data.groupby(['year', 'month'])  # year와 month를 기준으로 그룹화
    .apply(lambda x: x.nlargest(10, 'Predicted'))  # 'ret' 기준으로 상위 10개 선택
    .reset_index(drop=True)  # 그룹화 결과의 인덱스를 초기화
)

# 결과 출력
COR_top10_monthly

C:\Users\DaBin\AppData\Local\Temp\ipykernel_22620\2398869912.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.nlargest(10, 'Predicted'))  # 'ret' 기준으로 상위 10개 선택


,month,year,ticker,Predicted
0,1,2017,GPN,0.054587
1,1,2017,QRVO,0.051487
2,1,2017,TMUS,0.028169
3,1,2017,NVDA,0.026450
4,1,2017,DPZ,0.026163
...,...,...,...,...
625,3,2022,BRO,0.077769
626,3,2022,GIS,0.075965
627,3,2022,VRSN,0.074427
628,3,2022,AMP,0.073869


### 1.3 Yahoo Finance에서 해당 회사의 시가 총액 가져오기

In [9]:

import yfinance as yf
import pandas as pd

# Yahoo Finance를 사용해 시가총액 가져오기
def get_market_cap(ticker, year, month):
    try:
        # 시작일과 종료일 계산
        start_date = f"{int(year)}-{int(month):02d}-01"
        end_date = pd.to_datetime(start_date) + pd.offsets.MonthEnd(0)

        # Yahoo Finance에서 주식 데이터 다운로드
        stock_data = yf.download(ticker, start=start_date, end=end_date, interval="1mo")
        
        # 가격 데이터가 비어 있는 경우 None 반환
        if stock_data.empty:
            print(f"No price data for {ticker} in {year}-{month}.")
            return None

        # 월말 종가 (값만 가져오기)
        last_close = stock_data["Close"].iloc[-1] if "Close" in stock_data else None

        # 발행 주식 수 가져오기
        ticker_info = yf.Ticker(ticker).info
        shares_outstanding = ticker_info.get("sharesOutstanding", None)

        # 시가총액 계산
        if last_close is not None and shares_outstanding is not None:
            market_cap = float(last_close) * shares_outstanding  # 값만 남기기
            return market_cap
        else:
            print(f"Missing data for {ticker}: Close={last_close}, Shares Outstanding={shares_outstanding}")
            return None
    except Exception as e:
        print(f"Error processing {ticker} ({year}-{month}): {e}")
        return None

# 새로운 데이터프레임 생성
results = []

# 주어진 데이터의 각 행에 대해 시가총액 가져오기
for _, row in COR_top10_monthly.iterrows():
    market_cap = get_market_cap(row["ticker"], row["year"], row["month"])
    results.append({
        "year": row["year"],
        "month": row["month"],
        "ticker": row["ticker"],
        "Market_Cap": market_cap
    })

# 결과를 데이터프레임으로 변환
cor_result_df = pd.DataFrame(results)

# 결과 출력
print(cor_result_df)


[*********************100%***********************]  1 of 1 completed
C:\Users\DaBin\AppData\Local\Temp\ipykernel_22620\725250970.py:28: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  market_cap = float(last_close) * shares_outstanding  # 값만 남기기
[*********************100%***********************]  1 of 1 completed
C:\Users\DaBin\AppData\Local\Temp\ipykernel_22620\725250970.py:28: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  market_cap = float(last_close) * shares_outstanding  # 값만 남기기
[*********************100%***********************]  1 of 1 completed
C:\Users\DaBin\AppData\Local\Temp\ipykernel_22620\725250970.py:28: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  market_cap = float(last_close) * 

     year  month ticker    Market_Cap
0    2017      1    GPN  1.966737e+10
1    2017      1   QRVO  6.069560e+09
2    2017      1   TMUS  7.226371e+10
3    2017      1   NVDA  6.684546e+10
4    2017      1    DPZ  6.027215e+09
..    ...    ...    ...           ...
625  2022      3    BRO  2.066618e+10
626  2022      3    GIS  3.759537e+10
627  2022      3   VRSN  2.137841e+10
628  2022      3    AMP  2.913930e+10
629  2022      3   AKAM  1.793560e+10

[630 rows x 4 columns]


C:\Users\DaBin\AppData\Local\Temp\ipykernel_22620\725250970.py:28: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  market_cap = float(last_close) * shares_outstanding  # 값만 남기기


In [13]:
cor_result_df['ticker'].unique()

array(['GPN', 'QRVO', 'TMUS', 'NVDA', 'DPZ', 'AMD', 'WAT', 'FITB', 'DG',
       'STE', 'SBAC', 'ALGN', 'GE', 'BLDR', 'ITW', 'FCX', 'A', 'NDAQ',
       'TGT', 'FICO', 'FIS', 'CSGP', 'MCD', 'ANET', 'ENPH', 'EXPE', 'LOW',
       'PYPL', 'DRI', 'PRU', 'EOG', 'NSC', 'NUE', 'GM', 'DECK', 'CMCSA',
       'GDDY', 'SYF', 'EQIX', 'CNC', 'PKG', 'MKTX', 'TSLA', 'AMT', 'CCI',
       'CPRT', 'WBA', 'USB', 'XYL', 'TRV', 'VRSN', 'NVR', 'INTC', 'AKAM',
       'ZTS', 'NEE', 'CME', 'OMC', 'LUV', 'LNT', 'JBHT', 'ICE', 'FANG',
       'TECH', 'KEYS', 'PWR', 'EXC', 'LKQ', 'MDLZ', 'VMC', 'HUBB', 'WTW',
       'BKNG', 'AMGN', 'LDOS', 'ES', 'PANW', 'MKC', 'NI', 'EA', 'LEN',
       'COF', 'UAL', 'HRL', 'WM', 'D', 'HBAN', 'BIIB', 'RL', 'ADSK',
       'BLK', 'DVN', 'AVGO', 'CTAS', 'PSX', 'IFF', 'VZ', 'TSN', 'CAH',
       'SBUX', 'DVA', 'V', 'GWW', 'HUM', 'DXCM', 'JBL', 'MRK', 'LVS',
       'KMI', 'PHM', 'TMO', 'MSCI', 'MDT', 'CVX', 'DELL', 'AMZN', 'SCHW',
       'REGN', 'ZBRA', 'ANSS', 'HII', 'QCOM', 'KR', 'INCY',

### 1.4 수익률 계산

In [ ]:
import pandas as pd
import numpy as np

# 초기 투자 금액
initial_investment = 10000

# 투자 시뮬레이션 데이터프레임 생성
investment_results = []

# 2017년 1월부터 2022년 2월까지 월별로 루프
start_year = 2017
start_month = 1
end_year = 2022
end_month = 3

# 투자 금액 초기화
current_investment = initial_investment

# 월별 반복 루프
for year in range(start_year, end_year + 1):
    for month in range(1, 13):
        # 종료 조건
        if year == end_year and month > end_month:
            break
        
        # data에서 해당 연도와 월의 상위 티커 필터링
        filtered_data = data[(data['year'] == year) & (data['month'] == month)]

        # 해당 월에 투자할 티커 목록 가져오기
        tickers = filtered_data['ticker'].tolist()

        # filtered_firm_data에서 해당 연도와 월에 대한 수익률 데이터 필터링
        returns_data = filtered_firm_data[
            (filtered_firm_data['year'] == year) &
            (filtered_firm_data['month'] == month) &
            (filtered_firm_data['ticker'].isin(tickers))
        ]

        # 만약 해당 월에 투자할 티커 데이터가 없으면 넘어감
        if returns_data.empty:
            investment_results.append({
                'year': year,
                'month': month,
                'investment': current_investment,
                'return_rate': None,
                'updated_investment': current_investment
            })
            continue

        # 투자 금액을 티커 개수만큼 균등 분배
        num_tickers = len(returns_data)
        investment_per_ticker = current_investment / num_tickers

        # 각 티커의 수익률 적용 후 총 투자 금액 계산
        returns_data['investment_value'] = investment_per_ticker * (1 + returns_data['ret'])
        updated_investment = returns_data['investment_value'].sum()

        # 수익률 계산
        return_rate = (updated_investment - current_investment) / current_investment * 100

        # 결과 저장
        investment_results.append({
            'year': year,
            'month': month,
            'investment': current_investment,
            'return_rate': return_rate,
            'updated_investment': updated_investment
        })

        # 현재 투자 금액 업데이트
        current_investment = updated_investment

# 결과를 데이터프레임으로 변환
investment_results_df = pd.DataFrame(investment_results)

# 결과 출력
investment_results_df


C:\Users\DaBin\AppData\Local\Temp\ipykernel_15972\3409622226.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  returns_data['investment_value'] = investment_per_ticker * (1 + returns_data['ret'])
C:\Users\DaBin\AppData\Local\Temp\ipykernel_15972\3409622226.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  returns_data['investment_value'] = investment_per_ticker * (1 + returns_data['ret'])
C:\Users\DaBin\AppData\Local\Temp\ipykernel_15972\3409622226.py:55: SettingWithCopyWarning: 
A value is trying to 

,year,month,investment,return_rate,updated_investment
0,2017,1,10000.000000,3.284875,10328.487456
1,2017,2,10328.487456,0.456782,10375.666106
2,2017,3,10375.666106,-3.067616,10057.380535
3,2017,4,10057.380535,0.049978,10062.407026
4,2017,5,10062.407026,1.445375,10207.846534
...,...,...,...,...,...
57,2021,10,21774.283797,6.061800,23094.197431
58,2021,11,23094.197431,-1.448960,22759.571760
59,2021,12,22759.571760,7.219157,24402.620939
60,2022,1,24402.620939,-8.278051,22382.559618


## 2. Category Data

### 2.1 예측 수익률 데이터 + 실제 회사의 수익률 데이터 가져오기

In [14]:
import pandas as pd

data = pd.read_csv('C:/Users/DaBin/Desktop/대학원/2024-2/통계계산특론/PROJECT/result/firm10_macro80_pred.csv')
filtered_firm_data = pd.read_csv('C:/Users/DaBin/Desktop/대학원/2024-2/통계계산특론/PROJECT/data/firm_new.csv')

### 2.2 월별 예측 수익률 top10 선택

In [15]:
# 월별 수익률 top10 선택
category_top10_monthly = (
    data.groupby(['year', 'month'])  # year와 month를 기준으로 그룹화
    .apply(lambda x: x.nlargest(10, 'Predicted'))  # 'ret' 기준으로 상위 10개 선택
    .reset_index(drop=True)  # 그룹화 결과의 인덱스를 초기화
)

# 결과 출력
category_top10_monthly

C:\Users\DaBin\AppData\Local\Temp\ipykernel_22620\3340465350.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.nlargest(10, 'Predicted'))  # 'ret' 기준으로 상위 10개 선택


,Predicted,month,year,ticker
0,0.058000,1,2017,NSC
1,0.056277,1,2017,DG
2,0.056262,1,2017,DVN
3,0.055665,1,2017,HII
4,0.054431,1,2017,LDOS
...,...,...,...,...
625,0.034245,3,2022,CRL
626,0.034160,3,2022,DVN
627,0.033711,3,2022,ADBE
628,0.033368,3,2022,FSLR


### 2.3 Yahoo Finance에서 해당 회사의 시가 총액 가져오기

In [17]:

import yfinance as yf
import pandas as pd

# Yahoo Finance를 사용해 시가총액 가져오기
def get_market_cap(ticker, year, month):
    try:
        # 시작일과 종료일 계산
        start_date = f"{int(year)}-{int(month):02d}-01"
        end_date = pd.to_datetime(start_date) + pd.offsets.MonthEnd(0)

        # Yahoo Finance에서 주식 데이터 다운로드
        stock_data = yf.download(ticker, start=start_date, end=end_date, interval="1mo")
        
        # 가격 데이터가 비어 있는 경우 None 반환
        if stock_data.empty:
            print(f"No price data for {ticker} in {year}-{month}.")
            return None

        # 월말 종가 (값만 가져오기)
        last_close = stock_data["Close"].iloc[-1] if "Close" in stock_data else None

        # 발행 주식 수 가져오기
        ticker_info = yf.Ticker(ticker).info
        shares_outstanding = ticker_info.get("sharesOutstanding", None)

        # 시가총액 계산
        if last_close is not None and shares_outstanding is not None:
            market_cap = float(last_close) * shares_outstanding  # 값만 남기기
            return market_cap
        else:
            print(f"Missing data for {ticker}: Close={last_close}, Shares Outstanding={shares_outstanding}")
            return None
    except Exception as e:
        print(f"Error processing {ticker} ({year}-{month}): {e}")
        return None

# 새로운 데이터프레임 생성
results = []

# 주어진 데이터의 각 행에 대해 시가총액 가져오기
for _, row in category_top10_monthly.iterrows():
    market_cap = get_market_cap(row["ticker"], row["year"], row["month"])
    results.append({
        "year": row["year"],
        "month": row["month"],
        "ticker": row["ticker"],
        "Market_Cap": market_cap
    })

# 결과를 데이터프레임으로 변환
category_result_df = pd.DataFrame(results)

# 결과 출력
print(category_result_df)


[*********************100%***********************]  1 of 1 completed
C:\Users\DaBin\AppData\Local\Temp\ipykernel_22620\1800712565.py:28: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  market_cap = float(last_close) * shares_outstanding  # 값만 남기기
[*********************100%***********************]  1 of 1 completed
C:\Users\DaBin\AppData\Local\Temp\ipykernel_22620\1800712565.py:28: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  market_cap = float(last_close) * shares_outstanding  # 값만 남기기
[*********************100%***********************]  1 of 1 completed
C:\Users\DaBin\AppData\Local\Temp\ipykernel_22620\1800712565.py:28: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  market_cap = float(last_close)

     year  month ticker    Market_Cap
0    2017      1    GPN  1.966737e+10
1    2017      1   QRVO  6.069560e+09
2    2017      1   TMUS  7.226371e+10
3    2017      1   NVDA  6.684546e+10
4    2017      1    DPZ  6.027215e+09
..    ...    ...    ...           ...
625  2022      3    BRO  2.066618e+10
626  2022      3    GIS  3.759537e+10
627  2022      3   VRSN  2.137841e+10
628  2022      3    AMP  2.913930e+10
629  2022      3   AKAM  1.793560e+10

[630 rows x 4 columns]


C:\Users\DaBin\AppData\Local\Temp\ipykernel_22620\1800712565.py:28: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  market_cap = float(last_close) * shares_outstanding  # 값만 남기기


In [19]:
category_result_df['ticker'].nunique()

139

### 2.4 수익률 계산

## 3. Original Data

### 3.1 예측 수익률 데이터 + 실제 회사의 수익률 데이터 가져오기

In [20]:
import pandas as pd

data = pd.read_csv('C:/Users/DaBin/Desktop/대학원/2024-2/통계계산특론/PROJECT/result/S_PCA58F_PCA50E_pred.csv')
filtered_firm_data = pd.read_csv('C:/Users/DaBin/Desktop/대학원/2024-2/통계계산특론/PROJECT/data/firm_new.csv')

### 3.2 월별 예측 수익률 top10 선택

In [22]:
# 월별 수익률 top10 선택
top10_monthly = (
    data.groupby(['year', 'month'])  # year와 month를 기준으로 그룹화
    .apply(lambda x: x.nlargest(10, 'ret'))  # 'ret' 기준으로 상위 10개 선택
    .reset_index(drop=True)  # 그룹화 결과의 인덱스를 초기화
)

# 결과 출력
top10_monthly

C:\Users\DaBin\AppData\Local\Temp\ipykernel_22620\2787277609.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.nlargest(10, 'ret'))  # 'ret' 기준으로 상위 10개 선택


,ret,year,month,ticker
0,0.101890,2017.0,1.0,WTW
1,0.073279,2017.0,1.0,EQIX
2,0.063190,2017.0,1.0,ULTA
3,0.056740,2017.0,1.0,ROP
4,0.055307,2017.0,1.0,SBAC
...,...,...,...,...
625,0.065048,2022.0,3.0,FTNT
626,0.059239,2022.0,3.0,ENPH
627,0.057910,2022.0,3.0,STLD
628,0.056977,2022.0,3.0,CNP


### 3.3 Yahoo Finance에서 해당 회사의 시가 총액 가져오기

In [23]:

import yfinance as yf
import pandas as pd

# Yahoo Finance를 사용해 시가총액 가져오기
def get_market_cap(ticker, year, month):
    try:
        # 시작일과 종료일 계산
        start_date = f"{int(year)}-{int(month):02d}-01"
        end_date = pd.to_datetime(start_date) + pd.offsets.MonthEnd(0)

        # Yahoo Finance에서 주식 데이터 다운로드
        stock_data = yf.download(ticker, start=start_date, end=end_date, interval="1mo")
        
        # 가격 데이터가 비어 있는 경우 None 반환
        if stock_data.empty:
            print(f"No price data for {ticker} in {year}-{month}.")
            return None

        # 월말 종가 (값만 가져오기)
        last_close = stock_data["Close"].iloc[-1] if "Close" in stock_data else None

        # 발행 주식 수 가져오기
        ticker_info = yf.Ticker(ticker).info
        shares_outstanding = ticker_info.get("sharesOutstanding", None)

        # 시가총액 계산
        if last_close is not None and shares_outstanding is not None:
            market_cap = float(last_close) * shares_outstanding  # 값만 남기기
            return market_cap
        else:
            print(f"Missing data for {ticker}: Close={last_close}, Shares Outstanding={shares_outstanding}")
            return None
    except Exception as e:
        print(f"Error processing {ticker} ({year}-{month}): {e}")
        return None

# 새로운 데이터프레임 생성
results = []

# 주어진 데이터의 각 행에 대해 시가총액 가져오기
for _, row in top10_monthly.iterrows():
    market_cap = get_market_cap(row["ticker"], row["year"], row["month"])
    results.append({
        "year": row["year"],
        "month": row["month"],
        "ticker": row["ticker"],
        "Market_Cap": market_cap
    })

# 결과를 데이터프레임으로 변환
result_df = pd.DataFrame(results)

# 결과 출력
print(result_df)


[*********************100%***********************]  1 of 1 completed
C:\Users\DaBin\AppData\Local\Temp\ipykernel_22620\4102882079.py:28: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  market_cap = float(last_close) * shares_outstanding  # 값만 남기기
[*********************100%***********************]  1 of 1 completed
C:\Users\DaBin\AppData\Local\Temp\ipykernel_22620\4102882079.py:28: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  market_cap = float(last_close) * shares_outstanding  # 값만 남기기
[*********************100%***********************]  1 of 1 completed
C:\Users\DaBin\AppData\Local\Temp\ipykernel_22620\4102882079.py:28: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  market_cap = float(last_close)

       year  month ticker    Market_Cap
0    2017.0    1.0    WTW  1.260372e+10
1    2017.0    1.0   EQIX  3.714603e+10
2    2017.0    1.0   ULTA  1.262630e+10
3    2017.0    1.0    ROP  2.057188e+10
4    2017.0    1.0   SBAC  1.131787e+10
..      ...    ...    ...           ...
625  2022.0    3.0   FTNT  5.238553e+10
626  2022.0    3.0   ENPH  2.726189e+10
627  2022.0    3.0   STLD  1.270180e+10
628  2022.0    3.0    CNP  1.996891e+10
629  2022.0    3.0    XOM  3.629905e+11

[630 rows x 4 columns]


C:\Users\DaBin\AppData\Local\Temp\ipykernel_22620\4102882079.py:28: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  market_cap = float(last_close) * shares_outstanding  # 값만 남기기


In [25]:
result_df['ticker'].nunique()

251